In [1]:
# ============================================================
# Assignment 3: Defenses to Enhance CNN Robustness on CIFAR-10
# ============================================================

!pip install lpips
import os, torch, torch.nn as nn, torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Subset
from torchvision.utils import save_image
import numpy as np, pandas as pd
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score
import lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvi

In [2]:
# ------------------------------------------------------------
# Setup
# ------------------------------------------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTDIR = "./output_defenses"
os.makedirs(OUTDIR, exist_ok=True)
MEAN, STD = [0.4914,0.4822,0.4465], [0.247,0.243,0.261]

In [3]:
# ------------------------------------------------------------
# Utilities
# ------------------------------------------------------------
def denorm(x):
    mean=torch.tensor(MEAN).view(1,3,1,1).to(x.device)
    std=torch.tensor(STD).view(1,3,1,1).to(x.device)
    return torch.clamp(x*std+mean,0,1)

def get_loaders(bs=128):
    tf_train=transforms.Compose([
        transforms.RandomCrop(32,4), transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), transforms.Normalize(MEAN,STD)])
    tf_test=transforms.Compose([transforms.ToTensor(), transforms.Normalize(MEAN,STD)])
    tr=CIFAR10("./data",train=True,download=True,transform=tf_train)
    te=CIFAR10("./data",train=False,download=True,transform=tf_test)
    return tr,te,DataLoader(tr,bs,shuffle=True,num_workers=2),DataLoader(te,bs,shuffle=False,num_workers=2)

def create_resnet18(num_classes=10):
    m=torchvision.models.resnet18(weights=None)
    m.conv1=nn.Conv2d(3,64,3,1,1,bias=False); m.maxpool=nn.Identity()
    m.fc=nn.Linear(m.fc.in_features,num_classes)
    return m

def eval_acc(model,loader):
    model.eval(); c=0;t=0
    with torch.no_grad():
        for x,y in loader:
            x,y=x.to(DEVICE),y.to(DEVICE)
            c+=(model(x).argmax(1)==y).sum().item(); t+=y.size(0)
    return c/t

In [4]:
# ------------------------------------------------------------
# Baseline model
# ------------------------------------------------------------
def train_baseline(epochs=30,lr=0.1):
    print("\n🚀 Training baseline ResNet-18...")
    tr,te,tr_loader,te_loader=get_loaders()
    m=create_resnet18().to(DEVICE)
    opt=torch.optim.SGD(m.parameters(),lr=lr,momentum=0.9,weight_decay=5e-4)
    sch=torch.optim.lr_scheduler.CosineAnnealingLR(opt,T_max=epochs)
    for e in range(1,epochs+1):
        m.train()
        for x,y in tr_loader:
            x,y=x.to(DEVICE),y.to(DEVICE)
            opt.zero_grad(); F.cross_entropy(m(x),y).backward(); opt.step()
        sch.step()
        if e%5==0: print(f"Epoch {e}/{epochs}: acc={eval_acc(m,te_loader):.4f}")
    acc=eval_acc(m,te_loader)
    print(f"✅ Baseline done. Clean acc={acc:.4f}")
    torch.save(m.state_dict(),f"{OUTDIR}/baseline.pth")
    return m,tr,te,tr_loader,te_loader

In [5]:
# ------------------------------------------------------------
# FGSM Attack
# ------------------------------------------------------------
def fgsm(model,x,y,eps):
    x=x.clone().detach().to(DEVICE); x.requires_grad=True
    loss=F.cross_entropy(model(x),y.to(DEVICE))
    model.zero_grad(); loss.backward()
    adv=x+eps*x.grad.sign()
    return torch.clamp(adv,-5,5)

In [6]:
# ------------------------------------------------------------
# 1️⃣ Adversarial Defense: FGSM Adversarial Training
# ------------------------------------------------------------
def eval_adversarial(model,loader,eps):
    lp=lpips.LPIPS(net='alex').to(DEVICE)
    model.eval(); c=0;a=0;flip=0;t=0;lpv=[]
    for x,y in loader:
        x,y=x.to(DEVICE),y.to(DEVICE)
        pc=model(x).argmax(1)
        c+=(pc==y).sum().item()
        adv=fgsm(model,x,y,eps)
        mean,std=torch.tensor(MEAN).view(1,3,1,1).to(DEVICE),torch.tensor(STD).view(1,3,1,1).to(DEVICE)
        adv_norm=(adv-mean)/std
        pa=model(adv_norm).argmax(1)
        a+=(pa==y).sum().item()
        flip+=((pc==y)&(pa!=y)).sum().item()
        lpv.append(lp(denorm(x)*2-1,denorm(adv)*2-1).mean().item())
        t+=y.size(0)
    return c/t,a/t,flip/max(1,c),np.mean(lpv)

def adversarial_defense_FGSM(tr_loader,te_loader,eps_list=[4/255,8/255,12/255],epochs=10):
    print("\n🛡️ Adversarial Defense (FGSM Adv Training)")
    results=[]
    for eps in eps_list:
        print(f"\n--- Training with ε={eps:.5f} ---")
        m=create_resnet18().to(DEVICE)
        opt=torch.optim.SGD(m.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4)
        sch=torch.optim.lr_scheduler.CosineAnnealingLR(opt,T_max=epochs)
        for e in range(1,epochs+1):
            m.train()
            for x,y in tr_loader:
                x,y=x.to(DEVICE),y.to(DEVICE)
                adv=fgsm(m,x,y,eps)
                opt.zero_grad(); F.cross_entropy(m(adv),y).backward(); opt.step()
            sch.step()
        acc_c,acc_a,asr,lp=eval_adversarial(m,te_loader,eps)
        print(f"ε={eps:.5f} → Clean={acc_c:.3f}, Adv={acc_a:.3f}, ASR={asr:.3f}, LPIPS={lp:.4f}")
        results.append({"Epsilon":eps,"CleanAcc":acc_c,"AdvAcc":acc_a,"ASR":asr,"LPIPS":lp})
    pd.DataFrame(results).to_csv(f"{OUTDIR}/defense_adversarial.csv",index=False)

In [7]:
# ------------------------------------------------------------
# 2️⃣ Poisoning Defense: Spectral Signature Filtering
# ------------------------------------------------------------
def spectral_signature_defense(model,trainset,test_loader,frac_list=[0.01,0.03,0.05]):
    print("\n🧩 Poisoning Defense (Spectral Signature Filtering)")
    results=[]
    for frac in frac_list:
        print(f"\n--- Fraction={frac} ---")
        loader=DataLoader(trainset,batch_size=256,shuffle=False)
        feats,labels=[],[];hook_out=[]
        def hook(_,__,out): hook_out.append(out.detach().cpu())
        h=model.avgpool.register_forward_hook(hook)
        with torch.no_grad():
            for x,y in loader:
                x=x.to(DEVICE); hook_out.clear(); _=model(x)
                f=hook_out[0].view(x.size(0),-1); feats.append(f); labels.append(y)
        h.remove()
        feats=torch.cat(feats).numpy(); labels=torch.cat(labels).numpy()
        susp=[]
        for cls in np.unique(labels):
            idx=np.where(labels==cls)[0]; f=feats[idx]
            mu=f.mean(0,keepdims=True); centered=f-mu
            sv=TruncatedSVD(n_components=1,random_state=0).fit(centered).components_[0]
            proj=np.abs(centered@sv); k=int(len(idx)*frac)
            susp.extend(idx[np.argsort(-proj)[:k]])
        print(f"Detected {len(susp)} suspicious samples.")
        keep=np.setdiff1d(np.arange(len(trainset)),susp)
        filt_loader=DataLoader(Subset(trainset,keep.tolist()),128,shuffle=True)
        clean_model=create_resnet18().to(DEVICE)
        opt=torch.optim.SGD(clean_model.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4)
        for e in range(10):
            clean_model.train()
            for x,y in filt_loader:
                x,y=x.to(DEVICE),y.to(DEVICE)
                opt.zero_grad(); F.cross_entropy(clean_model(x),y).backward(); opt.step()
        acc=eval_acc(clean_model,test_loader)
        print(f"Fraction={frac:.2f} → Acc={acc:.4f}")
        results.append({"Fraction":frac,"FilteredAcc":acc,"Removed":len(susp)})
    pd.DataFrame(results).to_csv(f"{OUTDIR}/defense_poison.csv",index=False)

In [8]:
# ------------------------------------------------------------
# 3️⃣ Membership Inference Defense: Baseline vs Defended
# ------------------------------------------------------------
def compute_mi_auc(model,trainset,testset,temp=1.5):
    def confs(ds):
        loader=DataLoader(ds,128,shuffle=False); c=[]
        with torch.no_grad():
            for x,y in loader:
                p=F.softmax(model(x.to(DEVICE))/temp,1)
                c.append(p.max(1)[0].cpu().numpy())
        return np.concatenate(c)
    tr_conf,te_conf=confs(trainset),confs(testset)
    y_true=np.concatenate([np.ones(len(tr_conf)),np.zeros(len(te_conf))])
    y_score=np.concatenate([tr_conf,te_conf])
    return roc_auc_score(y_true,y_score)

def membership_inference_defense(tr_loader,te_loader,tr,te):
    print("\n🔒 Membership Inference Defense (Label Smoothing + Temp Scaling)")
    results=[]
    # Baseline
    base=create_resnet18().to(DEVICE)
    opt=torch.optim.SGD(base.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4)
    for e in range(5):
        base.train()
        for x,y in tr_loader:
            x,y=x.to(DEVICE),y.to(DEVICE)
            opt.zero_grad(); F.cross_entropy(base(x),y).backward(); opt.step()
    auc_base=compute_mi_auc(base,tr,te)
    print(f"Baseline MI AUC={auc_base:.3f}")
    results.append({"Model":"Baseline","AUC":auc_base})
    # Defended
    m=create_resnet18().to(DEVICE)
    opt=torch.optim.SGD(m.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4)
    sch=torch.optim.lr_scheduler.CosineAnnealingLR(opt,T_max=30); smooth=0.1
    for e in range(1,31):
        m.train()
        for x,y in tr_loader:
            x,y=x.to(DEVICE),y.to(DEVICE)
            opt.zero_grad()
            logits=m(x); logp=F.log_softmax(logits,1)
            k=logits.size(1)
            sm=torch.full((y.size(0),k),smooth/(k-1)).to(DEVICE)
            sm.scatter_(1,y.unsqueeze(1),1-smooth)
            loss=F.kl_div(logp,sm,reduction='batchmean')
            loss.backward(); opt.step()
        sch.step()
    auc_def=compute_mi_auc(m,tr,te)
    print(f"Defended MI AUC={auc_def:.3f}")
    results.append({"Model":"Defended","AUC":auc_def})
    pd.DataFrame(results).to_csv(f"{OUTDIR}/defense_mi.csv",index=False)

In [9]:
# ------------------------------------------------------------
# 4️⃣ Model Inversion Defense: Gradient Regularization sweep
# ------------------------------------------------------------
def model_inversion_defense(tr_loader,te_loader,lambda_list=[0.0,0.02,0.05,0.1]):
    print("\n🧠 Model Inversion Defense (Gradient Regularization)")
    results=[]
    for lam in lambda_list:
        print(f"\n--- λ={lam} ---")
        m=create_resnet18().to(DEVICE)
        opt=torch.optim.SGD(m.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4)
        for e in range(1,21):
            m.train()
            for x,y in tr_loader:
                x,y=x.to(DEVICE),y.to(DEVICE)
                x.requires_grad=True
                logits=m(x); loss=F.cross_entropy(logits,y)
                loss.backward(retain_graph=True)
                grad_pen=(x.grad.view(x.size(0),-1).norm(2,1)**2).mean()
                opt.zero_grad(); (loss+lam*grad_pen).backward(); opt.step()
        acc=eval_acc(m,te_loader)
        print(f"λ={lam:.2f} → Clean acc={acc:.4f}")
        results.append({"Lambda":lam,"CleanAcc":acc})
    # Generate sample inversion for strongest defense
    x=torch.rand(1,3,32,32,device=DEVICE,requires_grad=True)
    opt=torch.optim.Adam([x],lr=0.05)
    mean,std=torch.tensor(MEAN).view(1,3,1,1).to(DEVICE),torch.tensor(STD).view(1,3,1,1).to(DEVICE)
    for i in range(300):
        opt.zero_grad(); logits=m((x-mean)/std); loss=-logits[0,0]; loss.backward(); opt.step()
        with torch.no_grad(): x.clamp_(0,1)
    save_image(denorm(x),f"{OUTDIR}/inversion_defended.png")
    pd.DataFrame(results).to_csv(f"{OUTDIR}/defense_inversion.csv",index=False)
    print("✅ Saved inversion_defended.png")

In [10]:
# ------------------------------------------------------------
# MAIN
# ------------------------------------------------------------
if __name__=="__main__":
    baseline,tr,te,tr_loader,te_loader=train_baseline(epochs=30)
    adversarial_defense_FGSM(tr_loader,te_loader)
    spectral_signature_defense(baseline,tr,te_loader)
    membership_inference_defense(tr_loader,te_loader,tr,te)
    model_inversion_defense(tr_loader,te_loader)
    print("\n🎯 All parameter sweeps complete. Check ./output_defenses for CSVs and images.")


🚀 Training baseline ResNet-18...


100%|██████████| 170M/170M [00:09<00:00, 17.4MB/s] 


Epoch 5/30: acc=0.7086
Epoch 10/30: acc=0.7618
Epoch 15/30: acc=0.8725
Epoch 20/30: acc=0.8866
Epoch 25/30: acc=0.9258
Epoch 30/30: acc=0.9359
✅ Baseline done. Clean acc=0.9359

🛡️ Adversarial Defense (FGSM Adv Training)

--- Training with ε=0.01569 ---
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 181MB/s]  


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
ε=0.01569 → Clean=0.866, Adv=0.224, ASR=0.760, LPIPS=0.0001

--- Training with ε=0.03137 ---
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
ε=0.03137 → Clean=0.840, Adv=0.245, ASR=0.732, LPIPS=0.0005

--- Training with ε=0.04706 ---
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
ε=0.04706 → Clean=0.815, Adv=0.261, ASR=0.715, LPIPS=0.0012

🧩 Poisoning Defense (Spectral Signature Filtering)

--- Fraction=0.01 ---
Detected 500 suspicious samples.
Fraction=0.01 → Acc=0.7933

--- Fraction=0.03 ---
Detected 1500 suspicious samples.
Fraction=0.03 → Acc=0.7855

--- Fraction=0.05 ---
Detected 2500 suspicious samples.
Fraction=0.05 → Acc=0.7461

🔒 Membership Inference Defense (Label Smoo

In [11]:
import shutil
shutil.make_archive("/kaggle/working/output", "zip", "/kaggle/working/output_defenses")

'/kaggle/working/output.zip'